In [59]:
import importlib.util
import subprocess
import sys

def install_package(package_name):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

def check_package(package_name):
    spec = importlib.util.find_spec(package_name)
    if spec is None:
        print(f"{package_name} is not installed. Installing...")
        install_package(package_name)
        print(f"...ready.")
    else:
        print(f"{package_name} is already installed.")

check_package("pyfinance")
sys.path.append('..')

pyfinance is already installed.


In [60]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from pyfinance.options import BSM
import pandas as pd
from utils.tools import implied_vol, get_delta

import numpy as np

In [61]:
def norehedge(S1, S2, C1, C2, delta):
    diffRE = delta * (S2 - S1)
    diffOG = C2 - C1
    A0 = diffOG - diffRE
    # if(np.isnan(A0)):
    #     print('NaN encountered in norehedge with values:')
    #     print('S1:', S1)
    #     print('S2:', S2)
    #     print('C1:', C1)
    #     print('C2:', C2)
    #     print('delta:', delta)
    #     print()
    return A0

def rehedge(S1, S2, C1, C2, delta, r, T, E):
    diffRE = delta * (S2 - S1)
    diffOG = C2 - C1
    A0 = diffOG - diffRE
    #computing new volatility and delta
    sigma_new = implied_vol(C2, S2, E, r, T)
    delta_new = get_delta(S2, E, r, sigma_new, T)
    # if(np.isnan(A0) or np.isnan(delta_new)):
    #     print('NaN encountered in rehedge with values:')
    #     print('S1:', S1)
    #     print('S2:', S2)
    #     print('C1:', C1)
    #     print('C2:', C2)
    #     print('delta:', delta)
    #     print('r:', r)
    #     print('T:', T)
    #     print('E:', E)
    #     print('sigma_new:', sigma_new)
    #     print('delta_new:', delta_new)
    #     print()
    return delta_new, A0

In [62]:
df = pd.read_csv("../data/processed/IBM_2023-01-20_processed_45.csv")

S = df["S"].values #underlying
T = df["TTM"].values #time from maturity
C = df["C"].values #call price
r = df["r"].values #risk ratio
K = df["K"].values[0] #risk ratio

A = np.zeros(len(T), dtype=float)

sigma_0 = implied_vol(C[0], S[0], K, r[0], T[0])
delta0 = get_delta(S[0], K, r[0], sigma_0, T[0])

# print('delta_0:', delta0)
# print(S[0], r[0], T[0], K)
# print('sigma_0:', sigma_0)

#portfolio OP
OP0 = C[0]

#portfolio RE
RE0 = delta0 * S[0]

for i in range(1, len(T)-1):
    if i%2 != 0: #if odd day dont rehedge
        A[i] = norehedge(S[i-1], S[i], C[i-1], C[i], delta0)
    else: 
        delta0, A[i] = rehedge(S[i-1], S[i], C[i-1], C[i], delta0, r[i], T[i], K)

In [63]:
print(A)

[ 0.          0.1041399  -0.44961456  0.17873557 -0.06694361  0.34038222
 -0.46468483  0.63780213  0.54695529 -0.2125493  -0.08857068  0.00947859
  0.03494997 -0.28516047 -0.03516047  0.1538693  -0.24334336 -0.11567044
 -0.16092652 -0.26807419  0.17974338 -0.10749401 -0.09670884 -0.21965307
  0.06876767 -0.45701341 -0.25408469 -0.21516371  1.97759635 -0.04098605
  0.        ]


In [64]:
mse = np.sum(A**2) / (len(A) - 2)
print(mse)

0.20539872806035517


In [65]:
maturities = ['01-20', '02-17', '03-17', '04-21', '05-19', '06-16', '07-21', '08-18', '09-15', '10-20', '11-17', '12-15']
for mat in maturities:
    df = pd.read_csv(f"../data/processed/IBM_2023-{mat}_processed_45.csv")

    S = df["S"].values #underlying
    T = df["TTM"].values #time from maturity
    C = df["C"].values #call price
    r = df["r"].values #risk ratio
    K = df["K"].values[0] #risk ratio

    A = np.zeros(len(T), dtype=float)

    sigma_0 = implied_vol(C[0], S[0], K, r[0], T[0])
    delta0 = get_delta(S[0], K, r[0], sigma_0, T[0])

    # print('delta_0:', delta0)
    # print(S[0], r[0], T[0], K)
    # print('sigma_0:', sigma_0)

    #portfolio OP
    OP0 = C[0]

    #portfolio RE
    RE0 = delta0 * S[0]

    for i in range(1, len(T)-1):
        if i%2 != 0: #if odd day dont rehedge
            A[i] = norehedge(S[i-1], S[i], C[i-1], C[i], delta0)
        else: 
            delta0, A[i] = rehedge(S[i-1], S[i], C[i-1], C[i], delta0, r[i], T[i], K)
    print(f'Results for maturitiy: 2023-{mat}:')
    print('A:', A)
    mse = np.sum(A**2) / (len(A) - 2)
    print(mse)
    print()

Results for maturitiy: 2023-01-20:
A: [ 0.          0.1041399  -0.44961456  0.17873557 -0.06694361  0.34038222
 -0.46468483  0.63780213  0.54695529 -0.2125493  -0.08857068  0.00947859
  0.03494997 -0.28516047 -0.03516047  0.1538693  -0.24334336 -0.11567044
 -0.16092652 -0.26807419  0.17974338 -0.10749401 -0.09670884 -0.21965307
  0.06876767 -0.45701341 -0.25408469 -0.21516371  1.97759635 -0.04098605
  0.        ]
0.20539872806035517

Results for maturitiy: 2023-02-17:
A: [ 0.          0.08040583  0.07799553 -0.12189008  0.23265773 -0.34718727
 -0.17712492 -0.07896804 -0.19637632 -0.08047563  0.47818072  0.38479993
 -0.53769542 -0.16142159 -0.05905153 -0.18091906  0.64232973 -0.02684127
 -0.04790738 -0.01367905 -0.05662376  0.01213278 -0.00755921  0.01420069
 -0.01917337  0.03882563 -0.01129401  0.01712219 -0.02164117  0.02968594
 -0.03446083  0.01096716  0.        ]
0.04636125451193708

Results for maturitiy: 2023-03-17:
A: [ 0.00000000e+00 -2.45727857e-01  8.43160709e-02 -8.36417495e-